# KD Tree for Bottleneck TSP

In [1]:
%load_ext autoreload
%autoreload 2

## Get Spotify Tracks

In [2]:
import tekore as tk
from smoothify.features import construct_features
import numpy as np

### Obtaining an API Access Token

You'll need an access token to call the Spotify API.

Here's how to get a temporary token to play around with (expires in ~30 mins):

- Visit https://developer.spotify.com/console/get-playlist-tracks/
- Click 'Get Token'
- Select the following permissions:
    - `user-library-read`
    - `playlist-read-private`
    - `playlist-read-collaborative`
    - `playlist-modify-public`
    - `playlist-modify-private`
- Click 'Request Token'
- Copy the text in the 'OAuth Token' field

Paste the access token below

In [3]:
ACCESS_TOKEN = "BQAqbqBHRy1U3xfvQQpSD2u83mY4xSv8F3d8R5XKmPGISxmDG_nuaznWg79OT59PtFZ45MmgdTwP7qTay9VuuIc019LcegFsVKcTyuL1-eGBPCzMbLF3fthPbgyj5AXlfxls-ESHGCpObITxsaEpDSSlFlBoyObKQisDJ0Rc7Z3atJbFEVWHelKR5TRL-uB8AbatIQQAiaG4aoMsG13FOdXB2PYyxPu8LdCw9eSPAwUxzYFGKQI"
spotify = tk.Spotify(ACCESS_TOKEN, asynchronous=True, max_limits_on=True, chunked_on=True)
current_user = await spotify.current_user()

### Fetch tracks from current user's Liked Songs

In [4]:
saved_tracks_page = await spotify.saved_tracks()
track_list = [track.track async for track in spotify.all_items(saved_tracks_page)]
playlist_name = f"{current_user.display_name}'s Library"
is_playlist_public = False
is_playlist_collaborative = False
print(f"Fetched {len(track_list)} tracks")

Fetched 531 tracks


### Fetch tracks from a playlist

To read the tracks in a playlist, you'll need the playlist's ID.

You can find a playlist's ID in its URL, which looks like `https://open.spotify.com/playlist/{playlist_id}`

#### Getting a Playlist's URL

- Navigate to your playlist
- Under the playlist's title, click the triple-dot icon and select to `Share -> Copy link to playlist`


In [ ]:
PLAYLIST_ID = "78RmalWaNX12cJSmfVHlXL"
playlist = await spotify.playlist(PLAYLIST_ID)
track_list = [track.track async for track in spotify.all_items(playlist.tracks)]
playlist_name = playlist.name
is_playlist_public = playlist.public
is_playlist_collaborative = playlist.collaborative
print(f"Fetched {len(track_list)} tracks")

## Construct audio features

### Query Spotify API for audio features

In [5]:
# Extract track IDs so we can fetch additional info
track_id_list = [track.id for track in track_list]
# Get track audio features
track_features_list = await spotify.tracks_audio_features(track_id_list)

### Merge & normalize features

In [6]:
features_df = construct_features(audio_features_list=track_features_list)
features_df.head()

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,1.526728,-0.558664,-0.349908,0.130886,-0.678244,-0.738907,-0.702520,1.075576,-0.019850
1,-0.270316,1.331587,0.874591,0.340775,-0.564689,-0.784524,-0.357060,1.306981,0.268654
2,0.836608,0.242758,0.155296,-0.756161,-0.296462,-0.784524,1.909257,0.596806,0.184870
3,0.440302,1.187884,0.869561,1.564870,-0.644971,-0.784497,1.063982,0.405299,0.104714
4,0.993764,-0.354163,0.046521,-0.474267,-0.070465,-0.515523,-0.516315,-1.098835,-0.020427


## Compute the best path through the points

In [7]:
import random
from uuid import uuid4
from functools import partial
from typing import List, Tuple

import pandas as pd
import scipy.spatial
from scipy.spatial import cKDTree
from joblib import Parallel, delayed

from tqdm import tqdm

from smoothify.optim.bottleneck_tsp import KDTreeBottleneckTSP

In [8]:
points = np.array(features_df)

In [9]:
optimizer = KDTreeBottleneckTSP(points=points)
results = optimizer.get_best_path()
best_path = results.best_path
max_dist = results.min_max_dist
print(f"Max edge length: {max_dist}")

  0%|                                                                                                                                                                                                                                 | 0/531 [00:00<?, ?it/s]

[ProgressParallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 531/531 [00:20<00:00, 26.22it/s]

[ProgressParallel(n_jobs=-1)]: Done 531 out of 531 | elapsed:   20.2s finished
Max edge length: 4.7158256001683645


## Create a smoothified playlist

**NOTE: Running this cell will add a playlist to your Spotify account**

In [ ]:
# Create new playlist
NEW_PLAYLIST_NAME = f"{playlist_name} (but smoother)"
new_playlist = await spotify.playlist_create(current_user.id, NEW_PLAYLIST_NAME, public=is_playlist_public)
await spotify.playlist_change_details(new_playlist.id, collaborative=is_playlist_collaborative)

# Add tracks to new playlist
smoothified_track_uris = [track_list[node_idx].uri for node_idx in best_path]
await spotify.playlist_add(new_playlist.id, smoothified_track_uris)

## Visualize paths

In [10]:
from sklearn.manifold import TSNE

In [11]:
reducer = TSNE(learning_rate="auto", init="pca")
points_2d = reducer.fit_transform(points)

/home/krishna/.pyenv/versions/3.10.0/envs/smoothify/lib/python3.10/site-packages/sklearn/manifold/_t_sne.py:982: FutureWarning: The PCA initialization in TSNE will change to have the standard deviation of PC1 equal to 1e-4 in 1.2. This will ensure better convergence.
  warnings.warn(


In [12]:
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource, HoverTool


def make_plot(*, x, y, path):
    p = figure(tools="hover")
    # Data source
    data = dict(x=x, y=y)
    # Hover tooltips
    hover = p.select({"type": HoverTool})
    hover.tooltips = {k: f"@{k}" for k in data}
    # Draw points
    p.scatter(source=ColumnDataSource(data=data), x="x", y="y", radius=0.5, fill_alpha=0.5, line_color=None)
    # Draw path
    p.line(x[path], y[path], alpha=0.33)
    return p


initial_order = make_plot(x=points_2d[:, 0], y=points_2d[:, 1], path=list(range(len(points_2d))))
best_order = make_plot(x=points_2d[:, 0], y=points_2d[:, 1], path=best_path)

output_notebook()
show(initial_order)
show(best_order)

Loading BokehJS ...